# MassBalanceMachine XGBoost Model Training - Example for the Iceland Region

In [1]:
import massbalancemachine as mbm
from sklearn.model_selection import GridSearchCV
import pandas as pd
import sklearn
import numpy as np

In [2]:
dataset = pd.read_csv('./example_data/iceland/files/region_monthly.csv')

In [3]:
# Select features for training
df_X_train = dataset.drop(['YEAR', 'POINT_BALANCE'], axis=1)

# Move id and n_months to the end of the dataframe (these are used as metadata)
# Columns to move to the end
metadata_columns = ['ID', 'N_MONTHS', 'MONTH']

# Reindex the DataFrame, moving the specified columns to the end
df_X_train = df_X_train[[col for col in df_X_train.columns if col not in metadata_columns] + metadata_columns]

# Select the targets for training
df_y_train = dataset[['POINT_BALANCE']]

# Get arrays of features+metadata and targets
X_train, y_train = df_X_train.values, df_y_train.values

# Get glacier IDs from training dataset (in the order of which they appear in training dataset).
# gp_s is an array with shape equal to the shape of X_train_s and y_train_s.
glacier_ids = np.array(dataset['ID'].values)

# Use five folds
group_kf_s = sklearn.model_selection.GroupKFold(n_splits=5)

# Split into folds according to group by glacier ID.
# For each unique glacier ID, indices in gp_s indicate which rows in X_train_s and y_train_s belong to the glacier.
splits = list(group_kf_s.split(X_train, y_train, glacier_ids))

In [4]:
# Create a base estimator with full metadata
base_estimator = mbm.models.xgboost.CustomXGBoostRegressor()

parameters = {
    'max_depth': [3, 4, 5,],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'gamma':[0, 1]
}

clf = GridSearchCV(estimator=base_estimator,
                   param_grid=parameters,
                   cv=splits,
                   verbose=10, 
                   n_jobs=-1, 
                   refit=True, 
                   return_train_score=True,
                   error_score='raise')

# Fit using features
clf.fit(df_X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=[(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 144, 145, 146, 147...
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              objective='reg:squarederror', ...),
             n_jobs=-1,
             param_grid={'gamma': [0, 1], 'learning_rate': [0.01, 0.1],
                         'max_depth': [3, 4, 5], 'n_estimators': [100, 200]},
             return_train_score=True, verbose=10)